É uma sequência de camadas e podemos usar o modelo sequencial oferecido pelo Keras, que possui as funções necessárias para construir cada camada de uma rede neural convolucional

O Keras utiliza o TensorFlow como backend, pois, na prática, O Keras é apenas uma biblioteca para simplificar a complexidade do TensorFlow. Aqui estão as versões utilizadas:

In [37]:
#Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Inicializando a rede neural convolucional

In [38]:
classifier = Sequential()

Defininco parâmetros para o shape dos dados de entrada e a função de ativação. Usaremos 32 features para um array 2D e definiremos nosso array como o formato 3x3

Converteremos todas as nossas imagens 64x64 pixels em um array 3D (pois as imagens são coloridas com 3 canais de cores)

In [39]:
#Passo 1 - Primeira Camada de Convolução
classifier.add(
    Conv2D(32, (3,3),input_shape = 
           (64,64,3), activation = 'relu'))

Em seguida, aplicamos o agrupamento (pooling) para reduzir o tamanho do mapa de features, resultado da primeria camada de convolução (dividido por 2):

In [40]:
#Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

Adicionamos, então, a segunda camada de convolução, tornando nossa rede um pouco mais profunda:

In [41]:
#Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3,3), 
                      activation = 'relu'))

Mais uma vez, aplicamos a camada de pooling à saída da camada de convolução anterior

In [42]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

Agora, aplicamos o "achatamento", ou apenas Flatten, para converter a estrutura de dados 2D, resultando da camada anterior em uma estrutura 1D, ou seja, um vetor

In [43]:
#Passo 3 - Flattening
classifier.add(Flatten())

No próximo passo, conectamos todas as camadas. Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmoide, para obter as probabilidados de cada imagem conter um cachorro ou um gato. O modelo raramente terá 100% de certeza e o que ele gera como resultado é uma probabilidade.

In [44]:
#Passo 4 - Full connection 
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

Finalmente, compilamos nossa rede neural. Para compilar a rede, usamos o otimizador "Adam", um excelente algoritmo para otimização baseada em gradiente de funções objetivas, estocásticas, que toma como base uma estimativa adaptada de momneto de baixa ordem.

Usamos uma função log loss com "entropia binária cruzada", pois ela funciona bem com funções sigmoides. Nossa métrica será a acurácia, pois essa é nossa maior preocupação no treinamento desse tipo de modelo.

In [45]:
#Compilando a rede
classifier.compile(optimizer = 'adam', 
                   loss = 'binary_crossen_tropy',
                   metrics = ['accuracy'])

Nesse ponto, temos nossa rede contruída. Precisamos, agora, treiná-la

### Treinando a rede neural convolucional

### Pré-processamento

Para essa tarefa, vamos usar a função "ImageDataGenerator()" do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

In [46]:
#Criando os objetos train_datagen e 
#validation_datagen com as regras de 
#pré-processamento das imagens

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale = 1./255)

Aplicamos, então, os dois objetos criados anteriormente para pré-processar os dados de treino e validação. Lembre-se: o tratamento aplicado aos dados de validação deve ser o mesmo aplicado aos dados de treino.

In [53]:
#Pré-processamento das imagens de treino e validação
training_set = train_datagen.flow_from_directory('/media/mello/Backup/Datasets/dogs-vs-cats/',
                                                 target_size=(64,64),
                                                 batch_size=32,
                                                 class_mode='binary'
                                                 )
validation_set = validation_datagen.flow_from_directory('/media/mello/Backup/Datasets/dogs-vs-cats/test/',
                                                        target_size=(64,64),
                                                        batch_size=32,
                                                        class_mode='binary'
                                                        )

Found 37500 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


### Treinamento

Usaremos 8.000 passos em nosso conjunto de treinamento para cada época. Escolhemos 2.000 etapas de validação para as imagens de validação - esses hiperparâmetros são definidos por você

In [48]:
#Executando o treinamento
classifier.fit(x = training_set,
                         steps_per_epoch = 8000,
                         epochs = 5,
                         validation_data = validation_set,
                         validation_steps = 2000)

ValueError: Asked to retrieve element 0, but the Sequence has length 0